Run this script to combine Land Use (2020), Zoning, and Building Permit Data to identify progress and remaining development potential in developing subdivisions

Data sources:

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Land_Use_2020/FeatureServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Building_Permit_Records/FeatureServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Planimetrics/FeatureServer

https://gis2.lawrenceks.org/ArcGIS/rest/services/Parcels/MapServer

https://tiles.arcgis.com/tiles/8O9UlSTnqjKptoda/arcgis/rest/services/LAWRENCE_KS_AERIAL_2020/MapServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Lawrence_Zoning/FeatureServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Plats/FeatureServer



In [1]:
#collect data sources

# import arcpy

# define variables
filegdb = r"C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum2021040201.gdb"


arcpy.conversion.FeatureClassToFeatureClass("Plats", filegdb, "PlatsCopy", "#", '#', '')
arcpy.conversion.FeatureClassToFeatureClass("Building Footprint", filegdb, "BuildingFootprints")
arcpy.management.AddField(filegdb+r"\PlatsCopy", "LotBlock", "TEXT", None, None, 24, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(filegdb+r"\PlatsCopy", "LotBlock", 'str(!BLOCK!)+"-"+ str(!LOT!)', "PYTHON3", '', "TEXT")
arcpy.management.FeatureToPoint("PlatsCopy", filegdb+r"\PlatsCopyPoints", "INSIDE")
arcpy.management.FeatureToPoint("Parcels", filegdb+r"\ParcelPoints", "INSIDE")
arcpy.conversion.FeatureClassToFeatureClass("Building Permit", filegdb, "NewPermitsResidential", "Issued >= timestamp '2018-01-01 00:00:00' And Classification IN ('COMMERCIAL NEW MULTI-FAMILY', 'RESIDENTIAL NEW DUPLEX', 'RESIDENTIAL NEW SINGLE FAMILY')", '#', '')
arcpy.analysis.SummarizeWithin("LU2020", filegdb+r"\BuildingFootprints", filegdb+r"\LandUseBuildings", "KEEP_ALL", "Shape_Area Sum", "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

# This is forming up "LandUseBuildingPermits_type01" which is eventually discarded in favor of type02
arcpy.analysis.SpatialJoin(filegdb+r"\LandUseBuildings",filegdb+r"\NewPermitsResidential", filegdb+r"\LandUseBuildingsPermits", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", None, '')
arcpy.management.FeatureToPoint("LandUseBuildingsPermits", filegdb+r"\LandUsePoints", "INSIDE")
arcpy.analysis.SpatialJoin("Lawrence Zoning", filegdb+r"\PlatsCopyPoints", filegdb+r"\PlatsZoning", "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.Dissolve("PlatsZoning", filegdb+r"\Subdivisions", "SUBDIVISION;NAME;Zoning", "Join_Count SUM;LotBlock COUNT", "MULTI_PART", "DISSOLVE_LINES")


<Result 'C:\\Users\\kgonterwitz\\Documents\\ArcGIS\\Projects\\Kyle Project 2\\BuildingSum2021040201.gdb\\Subdivisions'>

In [8]:
#replace all that with this -- this begins LandUseBuildingPermits_type02
filegdb = r"C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum2021040201.gdb"
arcpy.conversion.FeatureClassToFeatureClass("Building Permit", filegdb, "NewPermitsResidential", "Issued >= timestamp '2018-01-01 00:00:00' And Classification IN ('COMMERCIAL NEW MULTI-FAMILY', 'RESIDENTIAL NEW DUPLEX', 'RESIDENTIAL NEW SINGLE FAMILY')", '#', '')

# this is the call for LandUseBuildingPermits_type02
arcpy.analysis.SpatialJoin("LU2020",filegdb+r"\NewPermitsResidential", filegdb+r"\LandUseBuildingPermits", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", None, '')

# and here's the main call for AreaUnion
arcpy.analysis.Union("PlatsCopy #;LandUseBuildingPermits #;'Lawrence Zoning'", filegdb+r"\AreaUnion", "ALL", None, "GAPS")


<Result 'C:\\Users\\kgonterwitz\\Documents\\ArcGIS\\Projects\\Kyle Project 2\\BuildingSum2021040201.gdb\\Subdivisions'>

In [11]:
arcpy.management.Dissolve("AreaUnion",  filegdb+r"\Subdivisions", "SUBDIVISION;NAME;Zoning", "Issued MIN;Issued MAX;Issued MEAN;Submitted MIN;Submitted MAX", "MULTI_PART", "DISSOLVE_LINES")


<Result 'C:\\Users\\kgonterwitz\\Documents\\ArcGIS\\Projects\\Kyle Project 2\\BuildingSum2021040201.gdb\\Subdivisions'>

In [13]:

arcpy.analysis.SummarizeWithin(filegdb+r"\Subdivisions", filegdb+r"\ParcelPoints", filegdb+r"\Subdivisions_ParcelsWithin", "KEEP_ALL", "SYSCALACRES Sum", "ADD_SHAPE_SUM", '', None, "NO_MIN_MAJ", "NO_PERCENT")
#arcpy.management.AlterField(filegdb+r"\Subdivisions_ParcelsWithin", "SUM_Join_Count", "PlattedLots", "PlattedLots", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")
#arcpy.management.AlterField(filegdb+r"\Subdivisions_ParcelsWithin", "SUM_SYSCALACRES", "ParceledAcres", "ParceledAcres", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")
arcpy.analysis.SummarizeWithin(filegdb+r"\Subdivisions_ParcelsWithin", filegdb+r"\LandUsePoints", filegdb+r"\Subdivisions_Landuse", "KEEP_ALL", "SYSCALACRES Sum;FinalLand Sum;FinalBuilding Sum;FinalTotal Sum;flivunits Sum;Resyearbuilt Min;Resyearbuilt Max;Resyearbuilt Mean;SUM_Area_SQUAREFEET Sum;SUM_Area_SQUAREFEET Mean;FinalLand Mean;FinalBuilding Mean;FinalTotal Mean;flivunits Mean;FinalLand Stddev;FinalBuilding Stddev;FinalTotal Stddev;flivunits Stddev;Resyearbuilt Stddev", "ADD_SHAPE_SUM", '', None, "NO_MIN_MAJ", "NO_PERCENT", None)
arcpy.conversion.FeatureClassToFeatureClass("LandUsePoints", filegdb, "PermittedLandUseUnbuilt", "Permit IS NOT NULL And (Resyearbuilt IS NULL Or Generalization = 'Vacant')", '#', '')
arcpy.management.AddField( filegdb+r"\PermittedLandUseUnbuilt", "HousingUnits", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.SelectLayerByAttribute("PermittedLandUseUnbuilt", "NEW_SELECTION", "Classification = 'RESIDENTIAL NEW DUPLEX'", None)
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "HousingUnits", "2", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute("PermittedLandUseUnbuilt", "NEW_SELECTION", "Classification = 'RESIDENTIAL NEW SINGLE FAMILY'", None)
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "HousingUnits", "1", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute("PermittedLandUseUnbuilt", "NEW_SELECTION", "Classification = 'COMMERCIAL NEW MULTI-FAMILY'", None)
#arcpy.management.CalculateField("PermittedLandUseUnbuilt", "HousingUnits", "str(!Description![0:3]).isdigit()", "PYTHON3", '', "TEXT")
#just reviewed and edited these two, cant figure out how to extract count from description text
arcpy.management.AddField( filegdb+r"\PermittedLandUseUnbuilt", "PermitYearIssued", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.AddField( filegdb+r"\PermittedLandUseUnbuilt", "PermitYearSubmitted", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "PermitYearSubmitted", "str(!Submitted!)[:4]", "PYTHON3", '', "TEXT")
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "PermitYearIssued", "str(!Issued!)[:4]", "PYTHON3", '', "TEXT")
arcpy.analysis.SummarizeWithin( filegdb+r"\Subdivisions_Landuse",  filegdb+r"\PermittedLandUseUnbuilt",  filegdb+r"\Subdivisions", "KEEP_ALL", "SYSCALACRES Sum;Join_Count Sum;HousingUnits Sum;HousingUnits Mean;PermitYearSubmitted Min;PermitYearSubmitted Max;PermitYearIssued Min;PermitYearIssued Max;PermitYearIssued Mean;PermitYearSubmitted Mean", "ADD_SHAPE_SUM", '', None, "NO_MIN_MAJ", "NO_PERCENT", None)

# holy cow, this segment has so much code added on at the end. Maybe all this stuff is Environment variables?
LUBPvar = "\\GISFile\gisdata\Department\MSO\Projects\Internal\CHM_Gontwz_landuseSuitabilityRerun\Gonterwitz_Suitability_Rerun2022\Gonterwitz_Suitability_Rerun2022.gdb\LandUseBuildingPermits"
arcpy.conversion.FeatureClassToFeatureClass(filegdb+r"\LandUseBuildingsPermits", filegdb, "VacantUnpermitted", "Generalization = 'Vacant' And Permit IS NULL", r'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,LUBPvar,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,LUBPvar,TARGET_FID,-1,-1;JOINPIN "JOINPIN" true true false 23 Text 0 0,First,#,LUBPvar,JOINPIN,0,23;RECORDACRES "RECORDACRES" true true false 10 Text 0 0,First,#,LUBPvar,RECORDACRES,0,10;SYSCALACRES "SYSCALACRES" true true false 8 Double 0 0,First,#,LUBPvar,SYSCALACRES,-1,-1;PARCELNUMBER "PARCELNUMBER" true true false 6 Text 0 0,First,#,LUBPvar,PARCELNUMBER,0,6;owner1 "owner1" true true false 45 Text 0 0,First,#,LUBPvar,owner1,0,45;owner2 "owner2" true true false 45 Text 0 0,First,#,LUBPvar,owner2,0,45;owner3 "owner3" true true false 45 Text 0 0,First,#,LUBPvar,owner3,0,45;address "address" true true false 40 Text 0 0,First,#,LUBPvar,address,0,40;city "city" true true false 40 Text 0 0,First,#,LUBPvar,city,0,40;state "state" true true false 2 Text 0 0,First,#,LUBPvar,state,0,2;zip "zip" true true false 5 Text 0 0,First,#,LUBPvar,zip,0,5;plate "plate" true true false 12 Text 0 0,First,#,LUBPvar,plate,0,12;deed "deed" true true false 10 Text 0 0,First,#,LUBPvar,deed,0,10;book "book" true true false 4 Text 0 0,First,#,LUBPvar,book,0,4;page "page" true true false 4 Text 0 0,First,#,LUBPvar,page,0,4;txunit "txunit" true true false 3 Text 0 0,First,#,LUBPvar,txunit,0,3;school "school" true true false 3 Text 0 0,First,#,LUBPvar,school,0,3;legal "legal" true true false 3030 Text 0 0,First,#,LUBPvar,legal,0,3030;PID "PID" true true false 27 Text 0 0,First,#,LUBPvar,PID,0,27;Quickrefid "Quickrefid" true true false 20 Text 0 0,First,#,LUBPvar,Quickrefid,0,20;situs "situs" true true false 38 Text 0 0,First,#,LUBPvar,situs,0,38;AGMAPLINK "AGMAPLINK" true true false 150 Text 0 0,First,#,LUBPvar,AGMAPLINK,0,150;OBJECTID "OBJECTID" true true false 4 Long 0 0,First,#,LUBPvar,OBJECTID,-1,-1;PropertyNumber "PropertyNumber" true true false 255 Text 0 0,First,#,LUBPvar,PropertyNumber,0,255;AdHocTaxYear "AdHocTaxYear" true true false 8 Double 0 0,First,#,LUBPvar,AdHocTaxYear,-1,-1;fpropclass "fpropclass" true true false 255 Text 0 0,First,#,LUBPvar,fpropclass,0,255;FinalLand "FinalLand" true true false 8 Double 0 0,First,#,LUBPvar,FinalLand,-1,-1;FinalBuilding "FinalBuilding" true true false 8 Double 0 0,First,#,LUBPvar,FinalBuilding,-1,-1;FinalTotal "FinalTotal" true true false 8 Double 0 0,First,#,LUBPvar,FinalTotal,-1,-1;flivunits "flivunits" true true false 8 Double 0 0,First,#,LUBPvar,flivunits,-1,-1;Resyearbuilt "Resyearbuilt" true true false 8 Double 0 0,First,#,LUBPvar,Resyearbuilt,-1,-1;commerical_year_built "commerical year built" true true false 255 Text 0 0,First,#,LUBPvar,commerical_year_built,0,255;flbcsact "flbcsact" true true false 8 Double 0 0,First,#,LUBPvar,flbcsact,-1,-1;flbcsfunc "flbcsfunc" true true false 8 Double 0 0,First,#,LUBPvar,flbcsfunc,-1,-1;flbcsfuncsfx "flbcsfuncsfx" true true false 255 Text 0 0,First,#,LUBPvar,flbcsfuncsfx,0,255;flbcsown "flbcsown" true true false 8 Double 0 0,First,#,LUBPvar,flbcsown,-1,-1;flbcssite "flbcssite" true true false 8 Double 0 0,First,#,LUBPvar,flbcssite,-1,-1;JOINPIN_1 "JOINPIN" true true false 255 Text 0 0,First,#,LUBPvar,JOINPIN_1,0,255;Generalization "Generalization" true true false 150 Text 0 0,First,#,LUBPvar,Generalization,0,150;YearBuiltDate "YearBuiltDate" true true false 8 Date 0 0,First,#,LUBPvar,YearBuiltDate,-1,-1;SUM_Area_SQUAREFEET "Summarized area in SQUAREFEET" true true false 8 Double 0 0,First,#,LUBPvar,SUM_Area_SQUAREFEET,-1,-1;Permit "Permit" true true false 50 Text 0 0,First,#,LUBPvar,Permit,0,50;Address_1 "Address" true true false 100 Text 0 0,First,#,LUBPvar,Address_1,0,100;Classification "Classification" true true false 100 Text 0 0,First,#,LUBPvar,Classification,0,100;Submitted "Submitted" true true false 8 Date 0 0,First,#,LUBPvar,Submitted,-1,-1;Issued "Issued" true true false 8 Date 0 0,First,#,LUBPvar,Issued,-1,-1;status_1 "Status" true true false 20 Text 0 0,First,#,LUBPvar,status_1,0,20;Contractor "Contractor" true true false 100 Text 0 0,First,#,LUBPvar,Contractor,0,100;Description "Description" true true false 2147483647 Text 0 0,First,#,LUBPvar,Description,0,2147483647;pid_1 "pid" true true false 255 Text 0 0,First,#,LUBPvar,pid_1,0,255;X "X" true true false 8 Double 0 0,First,#,LUBPvar,X,-1,-1;Y "Y" true true false 8 Double 0 0,First,#,LUBPvar,Y,-1,-1;Xmin "Xmin" true true false 8 Double 0 0,First,#,LUBPvar,Xmin,-1,-1;Xmax "Xmax" true true false 8 Double 0 0,First,#,LUBPvar,Xmax,-1,-1;Ymin "Ymin" true true false 8 Double 0 0,First,#,LUBPvar,Ymin,-1,-1;Ymax "Ymax" true true false 8 Double 0 0,First,#,LUBPvar,Ymax,-1,-1;ARC_Single_Line_Input "Key" true true false 100 Text 0 0,First,#,LUBPvar,ARC_Single_Line_Input,0,100;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,LUBPvar,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,LUBPvar,Shape_Area,-1,-1', '')

# this one too. Gonna have to figure out which bits have been changed. 
arcpy.analysis.SpatialJoin(filegdb+r"\VacantUnpermitted", "Lawrence Zoning", filegdb+r"\VacantUnpermittedZoning", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,VacantUnpermitted,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,VacantUnpermitted,TARGET_FID,-1,-1;JOINPIN "JOINPIN" true true false 23 Text 0 0,First,#,VacantUnpermitted,JOINPIN,0,23;RECORDACRES "RECORDACRES" true true false 10 Text 0 0,First,#,VacantUnpermitted,RECORDACRES,0,10;SYSCALACRES "SYSCALACRES" true true false 8 Double 0 0,First,#,VacantUnpermitted,SYSCALACRES,-1,-1;PARCELNUMBER "PARCELNUMBER" true true false 6 Text 0 0,First,#,VacantUnpermitted,PARCELNUMBER,0,6;owner1 "owner1" true true false 45 Text 0 0,First,#,VacantUnpermitted,owner1,0,45;owner2 "owner2" true true false 45 Text 0 0,First,#,VacantUnpermitted,owner2,0,45;owner3 "owner3" true true false 45 Text 0 0,First,#,VacantUnpermitted,owner3,0,45;address "address" true true false 40 Text 0 0,First,#,VacantUnpermitted,address,0,40;city "city" true true false 40 Text 0 0,First,#,VacantUnpermitted,city,0,40;state "state" true true false 2 Text 0 0,First,#,VacantUnpermitted,state,0,2;zip "zip" true true false 5 Text 0 0,First,#,VacantUnpermitted,zip,0,5;plate "plate" true true false 12 Text 0 0,First,#,VacantUnpermitted,plate,0,12;deed "deed" true true false 10 Text 0 0,First,#,VacantUnpermitted,deed,0,10;book "book" true true false 4 Text 0 0,First,#,VacantUnpermitted,book,0,4;page "page" true true false 4 Text 0 0,First,#,VacantUnpermitted,page,0,4;txunit "txunit" true true false 3 Text 0 0,First,#,VacantUnpermitted,txunit,0,3;school "school" true true false 3 Text 0 0,First,#,VacantUnpermitted,school,0,3;legal "legal" true true false 3030 Text 0 0,First,#,VacantUnpermitted,legal,0,3030;PID "PID" true true false 27 Text 0 0,First,#,VacantUnpermitted,PID,0,27;Quickrefid "Quickrefid" true true false 20 Text 0 0,First,#,VacantUnpermitted,Quickrefid,0,20;situs "situs" true true false 38 Text 0 0,First,#,VacantUnpermitted,situs,0,38;AGMAPLINK "AGMAPLINK" true true false 150 Text 0 0,First,#,VacantUnpermitted,AGMAPLINK,0,150;OBJECTID "OBJECTID" true true false 4 Long 0 0,First,#,VacantUnpermitted,OBJECTID,-1,-1;PropertyNumber "PropertyNumber" true true false 255 Text 0 0,First,#,VacantUnpermitted,PropertyNumber,0,255;AdHocTaxYear "AdHocTaxYear" true true false 8 Double 0 0,First,#,VacantUnpermitted,AdHocTaxYear,-1,-1;fpropclass "fpropclass" true true false 255 Text 0 0,First,#,VacantUnpermitted,fpropclass,0,255;FinalLand "FinalLand" true true false 8 Double 0 0,First,#,VacantUnpermitted,FinalLand,-1,-1;FinalBuilding "FinalBuilding" true true false 8 Double 0 0,First,#,VacantUnpermitted,FinalBuilding,-1,-1;FinalTotal "FinalTotal" true true false 8 Double 0 0,First,#,VacantUnpermitted,FinalTotal,-1,-1;flivunits "flivunits" true true false 8 Double 0 0,First,#,VacantUnpermitted,flivunits,-1,-1;Resyearbuilt "Resyearbuilt" true true false 8 Double 0 0,First,#,VacantUnpermitted,Resyearbuilt,-1,-1;commerical_year_built "commerical year built" true true false 255 Text 0 0,First,#,VacantUnpermitted,commerical_year_built,0,255;flbcsact "flbcsact" true true false 8 Double 0 0,First,#,VacantUnpermitted,flbcsact,-1,-1;flbcsfunc "flbcsfunc" true true false 8 Double 0 0,First,#,VacantUnpermitted,flbcsfunc,-1,-1;flbcsfuncsfx "flbcsfuncsfx" true true false 255 Text 0 0,First,#,VacantUnpermitted,flbcsfuncsfx,0,255;flbcsown "flbcsown" true true false 8 Double 0 0,First,#,VacantUnpermitted,flbcsown,-1,-1;flbcssite "flbcssite" true true false 8 Double 0 0,First,#,VacantUnpermitted,flbcssite,-1,-1;JOINPIN_1 "JOINPIN" true true false 255 Text 0 0,First,#,VacantUnpermitted,JOINPIN_1,0,255;Generalization "Generalization" true true false 150 Text 0 0,First,#,VacantUnpermitted,Generalization,0,150;YearBuiltDate "YearBuiltDate" true true false 8 Date 0 0,First,#,VacantUnpermitted,YearBuiltDate,-1,-1;SUM_Area_SQUAREFEET "Summarized area in SQUAREFEET" true true false 8 Double 0 0,First,#,VacantUnpermitted,SUM_Area_SQUAREFEET,-1,-1;Permit "Permit" true true false 50 Text 0 0,First,#,VacantUnpermitted,Permit,0,50;Address_1 "Address" true true false 100 Text 0 0,First,#,VacantUnpermitted,Address_1,0,100;Classification "Classification" true true false 100 Text 0 0,First,#,VacantUnpermitted,Classification,0,100;Submitted "Submitted" true true false 8 Date 0 0,First,#,VacantUnpermitted,Submitted,-1,-1;Issued "Issued" true true false 8 Date 0 0,First,#,VacantUnpermitted,Issued,-1,-1;status_1 "Status" true true false 20 Text 0 0,First,#,VacantUnpermitted,status_1,0,20;Contractor "Contractor" true true false 100 Text 0 0,First,#,VacantUnpermitted,Contractor,0,100;Description "Description" true true false 2147483647 Text 0 0,First,#,VacantUnpermitted,Description,0,2147483647;pid_1 "pid" true true false 255 Text 0 0,First,#,VacantUnpermitted,pid_1,0,255;X "X" true true false 8 Double 0 0,First,#,VacantUnpermitted,X,-1,-1;Y "Y" true true false 8 Double 0 0,First,#,VacantUnpermitted,Y,-1,-1;Xmin "Xmin" true true false 8 Double 0 0,First,#,VacantUnpermitted,Xmin,-1,-1;Xmax "Xmax" true true false 8 Double 0 0,First,#,VacantUnpermitted,Xmax,-1,-1;Ymin "Ymin" true true false 8 Double 0 0,First,#,VacantUnpermitted,Ymin,-1,-1;Ymax "Ymax" true true false 8 Double 0 0,First,#,VacantUnpermitted,Ymax,-1,-1;ARC_Single_Line_Input "Key" true true false 100 Text 0 0,First,#,VacantUnpermitted,ARC_Single_Line_Input,0,100;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,VacantUnpermitted,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,VacantUnpermitted,Shape_Area,-1,-1;PUD_CLASS "PUD_CLASS" true true false 60 Text 0 0,First,#,Lawrence Zoning,PUD_CLASS,0,60;NAME "NAME" true true false 50 Text 0 0,First,#,Lawrence Zoning,NAME,0,50;Zoning "Zoning" true true false 15 Text 0 0,First,#,Lawrence Zoning,Zoning,0,15;GlobalID "GlobalID" false false true 38 GlobalID 0 0,First,#,Lawrence Zoning,GlobalID,-1,-1;OrdNum "Ordinance #" true true false 0 Short 0 0,First,#,Lawrence Zoning,OrdNum,-1,-1;created_user "created_user" false true false 255 Text 0 0,First,#,Lawrence Zoning,created_user,0,255;created_date "created_date" false true false 8 Date 0 0,First,#,Lawrence Zoning,created_date,-1,-1;last_edited_user "last_edited_user" false true false 255 Text 0 0,First,#,Lawrence Zoning,last_edited_user,0,255;last_edited_date "last_edited_date" false true false 8 Date 0 0,First,#,Lawrence Zoning,last_edited_date,-1,-1;Shape__Area "Shape__Area" false true true 0 Double 0 0,First,#,Lawrence Zoning,Shape__Area,-1,-1;Shape__Length "Shape__Length" false true true 0 Double 0 0,First,#,Lawrence Zoning,Shape__Length,-1,-1', "HAVE_THEIR_CENTER_IN", None, '')

# and then now, this one likely utilizes some of the bits from the above to orient the tool.
arcpy.conversion.FeatureClassToFeatureClass(filegdb+r"\VacantUnpermittedZoning", filegdb, "PotentialGrowthArea", "Zoning IS NOT NULL And NAME IN ('Multi-Dwelling Residential Office', 'Multi-Dwelling Residential', 'Duplex Multi-Dwelling Residential', 'Single-Dwelling Residential', 'Single-Dwelling Residential Office', 'Planned Residential Development', 'Planned Unit Development', 'Planned Commercial Development')", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,VacantUnpermittedZoning,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,VacantUnpermittedZoning,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,VacantUnpermittedZoning,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,VacantUnpermittedZoning,TARGET_FID_1,-1,-1;JOINPIN "JOINPIN" true true false 23 Text 0 0,First,#,VacantUnpermittedZoning,JOINPIN,0,23;RECORDACRES "RECORDACRES" true true false 10 Text 0 0,First,#,VacantUnpermittedZoning,RECORDACRES,0,10;SYSCALACRES "SYSCALACRES" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,SYSCALACRES,-1,-1;PARCELNUMBER "PARCELNUMBER" true true false 6 Text 0 0,First,#,VacantUnpermittedZoning,PARCELNUMBER,0,6;owner1 "owner1" true true false 45 Text 0 0,First,#,VacantUnpermittedZoning,owner1,0,45;owner2 "owner2" true true false 45 Text 0 0,First,#,VacantUnpermittedZoning,owner2,0,45;owner3 "owner3" true true false 45 Text 0 0,First,#,VacantUnpermittedZoning,owner3,0,45;address "address" true true false 40 Text 0 0,First,#,VacantUnpermittedZoning,address,0,40;city "city" true true false 40 Text 0 0,First,#,VacantUnpermittedZoning,city,0,40;state "state" true true false 2 Text 0 0,First,#,VacantUnpermittedZoning,state,0,2;zip "zip" true true false 5 Text 0 0,First,#,VacantUnpermittedZoning,zip,0,5;plate "plate" true true false 12 Text 0 0,First,#,VacantUnpermittedZoning,plate,0,12;deed "deed" true true false 10 Text 0 0,First,#,VacantUnpermittedZoning,deed,0,10;book "book" true true false 4 Text 0 0,First,#,VacantUnpermittedZoning,book,0,4;page "page" true true false 4 Text 0 0,First,#,VacantUnpermittedZoning,page,0,4;txunit "txunit" true true false 3 Text 0 0,First,#,VacantUnpermittedZoning,txunit,0,3;school "school" true true false 3 Text 0 0,First,#,VacantUnpermittedZoning,school,0,3;legal "legal" true true false 3030 Text 0 0,First,#,VacantUnpermittedZoning,legal,0,3030;PID "PID" true true false 27 Text 0 0,First,#,VacantUnpermittedZoning,PID,0,27;Quickrefid "Quickrefid" true true false 20 Text 0 0,First,#,VacantUnpermittedZoning,Quickrefid,0,20;situs "situs" true true false 38 Text 0 0,First,#,VacantUnpermittedZoning,situs,0,38;AGMAPLINK "AGMAPLINK" true true false 150 Text 0 0,First,#,VacantUnpermittedZoning,AGMAPLINK,0,150;OBJECTID "OBJECTID" true true false 4 Long 0 0,First,#,VacantUnpermittedZoning,OBJECTID,-1,-1;PropertyNumber "PropertyNumber" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,PropertyNumber,0,255;AdHocTaxYear "AdHocTaxYear" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,AdHocTaxYear,-1,-1;fpropclass "fpropclass" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,fpropclass,0,255;FinalLand "FinalLand" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,FinalLand,-1,-1;FinalBuilding "FinalBuilding" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,FinalBuilding,-1,-1;FinalTotal "FinalTotal" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,FinalTotal,-1,-1;flivunits "flivunits" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,flivunits,-1,-1;Resyearbuilt "Resyearbuilt" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Resyearbuilt,-1,-1;commerical_year_built "commerical year built" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,commerical_year_built,0,255;flbcsact "flbcsact" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,flbcsact,-1,-1;flbcsfunc "flbcsfunc" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,flbcsfunc,-1,-1;flbcsfuncsfx "flbcsfuncsfx" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,flbcsfuncsfx,0,255;flbcsown "flbcsown" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,flbcsown,-1,-1;flbcssite "flbcssite" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,flbcssite,-1,-1;JOINPIN_1 "JOINPIN" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,JOINPIN_1,0,255;Generalization "Generalization" true true false 150 Text 0 0,First,#,VacantUnpermittedZoning,Generalization,0,150;YearBuiltDate "YearBuiltDate" true true false 8 Date 0 0,First,#,VacantUnpermittedZoning,YearBuiltDate,-1,-1;SUM_Area_SQUAREFEET "Summarized area in SQUAREFEET" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,SUM_Area_SQUAREFEET,-1,-1;Permit "Permit" true true false 50 Text 0 0,First,#,VacantUnpermittedZoning,Permit,0,50;Address_1 "Address" true true false 100 Text 0 0,First,#,VacantUnpermittedZoning,Address_1,0,100;Classification "Classification" true true false 100 Text 0 0,First,#,VacantUnpermittedZoning,Classification,0,100;Submitted "Submitted" true true false 8 Date 0 0,First,#,VacantUnpermittedZoning,Submitted,-1,-1;Issued "Issued" true true false 8 Date 0 0,First,#,VacantUnpermittedZoning,Issued,-1,-1;status_1 "Status" true true false 20 Text 0 0,First,#,VacantUnpermittedZoning,status_1,0,20;Contractor "Contractor" true true false 100 Text 0 0,First,#,VacantUnpermittedZoning,Contractor,0,100;Description "Description" true true false 2147483647 Text 0 0,First,#,VacantUnpermittedZoning,Description,0,2147483647;pid_1 "pid" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,pid_1,0,255;X "X" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,X,-1,-1;Y "Y" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Y,-1,-1;Xmin "Xmin" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Xmin,-1,-1;Xmax "Xmax" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Xmax,-1,-1;Ymin "Ymin" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Ymin,-1,-1;Ymax "Ymax" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Ymax,-1,-1;ARC_Single_Line_Input "Key" true true false 100 Text 0 0,First,#,VacantUnpermittedZoning,ARC_Single_Line_Input,0,100;PUD_CLASS "PUD_CLASS" true true false 60 Text 0 0,First,#,VacantUnpermittedZoning,PUD_CLASS,0,60;NAME "NAME" true true false 50 Text 0 0,First,#,VacantUnpermittedZoning,NAME,0,50;Zoning "Zoning" true true false 15 Text 0 0,First,#,VacantUnpermittedZoning,Zoning,0,15;OrdNum "Ordinance #" true true false 2 Short 0 0,First,#,VacantUnpermittedZoning,OrdNum,-1,-1;created_user "created_user" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,created_user,0,255;created_date "created_date" true true false 8 Date 0 0,First,#,VacantUnpermittedZoning,created_date,-1,-1;last_edited_user "last_edited_user" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,last_edited_user,0,255;last_edited_date "last_edited_date" true true false 8 Date 0 0,First,#,VacantUnpermittedZoning,last_edited_date,-1,-1;Shape__Area "Shape__Area" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Shape__Area,-1,-1;Shape__Length "Shape__Length" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Shape__Length,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,VacantUnpermittedZoning,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,VacantUnpermittedZoning,Shape_Area,-1,-1', '')

arcpy.management.AddField("PotentialGrowthArea", "UnitPotential", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.SelectLayerByAttribute("PotentialGrowthArea", "NEW_SELECTION", "NAME IN ('Single-Dwelling Residential', 'Planned Residential Development', 'Single-Dwelling Residential Office')", None)
arcpy.management.CalculateField("PotentialGrowthArea", "UnitPotential", "1", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute("PotentialGrowthArea", "NEW_SELECTION", "UnitPotential IS NULL", None)
arcpy.management.CalculateField("PotentialGrowthArea", "UnitPotential", "2", "PYTHON3", '', "TEXT")


<Result 'PotentialGrowthArea'>

In [6]:
arcpy.conversion.FeatureClassToFeatureClass("AreaUnion", filegdb, "PermittedLandUseUnbuilt", "Permit IS NOT NULL And (Resyearbuilt IS NULL Or Generalization = 'Vacant')", '#', '')
arcpy.management.AddField( filegdb+r"\PermittedLandUseUnbuilt", "PotentialUnits", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.SelectLayerByAttribute("PermittedLandUseUnbuilt", "NEW_SELECTION", "Classification = 'RESIDENTIAL NEW DUPLEX'", None)
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "PotentialUnits", "2", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute("PermittedLandUseUnbuilt", "NEW_SELECTION", "Classification = 'RESIDENTIAL NEW SINGLE FAMILY'", None)
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "PotentialUnits", "1", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute("PermittedLandUseUnbuilt", "NEW_SELECTION", "Classification = 'COMMERCIAL NEW MULTI-FAMILY'", None)
#arcpy.management.CalculateField("PermittedLandUseUnbuilt", "HousingUnits", "str(!Description![0:3]).isdigit()", "PYTHON3", '', "TEXT")
#just reviewed and edited these two, cant figure out how to extract count from description text
arcpy.management.AddField( filegdb+r"\PermittedLandUseUnbuilt", "PermitYearIssued", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.AddField( filegdb+r"\PermittedLandUseUnbuilt", "PermitYearSubmitted", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "PermitYearSubmitted", "str(!Submitted!)[:4]", "PYTHON3", '', "TEXT")
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "PermitYearIssued", "str(!Issued!)[:4]", "PYTHON3", '', "TEXT")

# at this point, this is the second version of "permittedLandUseUnbuilt" ... so in my recreation of this, I called it "PermittedLandUseUnbuilt_v2"
arcpy.analysis.SummarizeWithin( filegdb+r"\Subdivisions",  filegdb+r"\PermittedLandUseUnbuilt",  filegdb+r"\Subdivisions_v3", "KEEP_ALL", "SYSCALACRES Sum;Join_Count Sum;PotentialUnits Sum;PotentialUnits Mean;PermitYearSubmitted Min;PermitYearSubmitted Max;PermitYearIssued Min;PermitYearIssued Max;PermitYearIssued Mean;PermitYearSubmitted Mean", "ADD_SHAPE_SUM", '', None, "NO_MIN_MAJ", "NO_PERCENT", None)
arcpy.conversion.FeatureClassToFeatureClass(filegdb+r"\LandUseBuildingsPermits", filegdb, "VacantUnpermitted", "Generalization = 'Vacant' And Permit IS NULL", r'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,TARGET_FID,-1,-1;JOINPIN "JOINPIN" true true false 23 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,JOINPIN,0,23;RECORDACRES "RECORDACRES" true true false 10 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,RECORDACRES,0,10;SYSCALACRES "SYSCALACRES" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,SYSCALACRES,-1,-1;PARCELNUMBER "PARCELNUMBER" true true false 6 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,PARCELNUMBER,0,6;owner1 "owner1" true true false 45 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,owner1,0,45;owner2 "owner2" true true false 45 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,owner2,0,45;owner3 "owner3" true true false 45 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,owner3,0,45;address "address" true true false 40 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,address,0,40;city "city" true true false 40 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,city,0,40;state "state" true true false 2 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,state,0,2;zip "zip" true true false 5 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,zip,0,5;plate "plate" true true false 12 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,plate,0,12;deed "deed" true true false 10 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,deed,0,10;book "book" true true false 4 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,book,0,4;page "page" true true false 4 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,page,0,4;txunit "txunit" true true false 3 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,txunit,0,3;school "school" true true false 3 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,school,0,3;legal "legal" true true false 3030 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,legal,0,3030;PID "PID" true true false 27 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,PID,0,27;Quickrefid "Quickrefid" true true false 20 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Quickrefid,0,20;situs "situs" true true false 38 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,situs,0,38;AGMAPLINK "AGMAPLINK" true true false 150 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,AGMAPLINK,0,150;OBJECTID "OBJECTID" true true false 4 Long 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,OBJECTID,-1,-1;PropertyNumber "PropertyNumber" true true false 255 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,PropertyNumber,0,255;AdHocTaxYear "AdHocTaxYear" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,AdHocTaxYear,-1,-1;fpropclass "fpropclass" true true false 255 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,fpropclass,0,255;FinalLand "FinalLand" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,FinalLand,-1,-1;FinalBuilding "FinalBuilding" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,FinalBuilding,-1,-1;FinalTotal "FinalTotal" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,FinalTotal,-1,-1;flivunits "flivunits" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,flivunits,-1,-1;Resyearbuilt "Resyearbuilt" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Resyearbuilt,-1,-1;commerical_year_built "commerical year built" true true false 255 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,commerical_year_built,0,255;flbcsact "flbcsact" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,flbcsact,-1,-1;flbcsfunc "flbcsfunc" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,flbcsfunc,-1,-1;flbcsfuncsfx "flbcsfuncsfx" true true false 255 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,flbcsfuncsfx,0,255;flbcsown "flbcsown" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,flbcsown,-1,-1;flbcssite "flbcssite" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,flbcssite,-1,-1;JOINPIN_1 "JOINPIN" true true false 255 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,JOINPIN_1,0,255;Generalization "Generalization" true true false 150 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Generalization,0,150;YearBuiltDate "YearBuiltDate" true true false 8 Date 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,YearBuiltDate,-1,-1;SUM_Area_SQUAREFEET "Summarized area in SQUAREFEET" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,SUM_Area_SQUAREFEET,-1,-1;Permit "Permit" true true false 50 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Permit,0,50;Address_1 "Address" true true false 100 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Address_1,0,100;Classification "Classification" true true false 100 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Classification,0,100;Submitted "Submitted" true true false 8 Date 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Submitted,-1,-1;Issued "Issued" true true false 8 Date 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Issued,-1,-1;status_1 "Status" true true false 20 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,status_1,0,20;Contractor "Contractor" true true false 100 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Contractor,0,100;Description "Description" true true false 2147483647 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Description,0,2147483647;pid_1 "pid" true true false 255 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,pid_1,0,255;X "X" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,X,-1,-1;Y "Y" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Y,-1,-1;Xmin "Xmin" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Xmin,-1,-1;Xmax "Xmax" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Xmax,-1,-1;Ymin "Ymin" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Ymin,-1,-1;Ymax "Ymax" true true false 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Ymax,-1,-1;ARC_Single_Line_Input "Key" true true false 100 Text 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,ARC_Single_Line_Input,0,100;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb\LandUseBuildingsPermits,Shape_Area,-1,-1', '')
#arcpy.analysis.SpatialJoin(filegdb+r"\VacantUnpermitted", "Lawrence Zoning", filegdb+r"\VacantUnpermittedZoning", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,VacantUnpermitted,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,VacantUnpermitted,TARGET_FID,-1,-1;JOINPIN "JOINPIN" true true false 23 Text 0 0,First,#,VacantUnpermitted,JOINPIN,0,23;RECORDACRES "RECORDACRES" true true false 10 Text 0 0,First,#,VacantUnpermitted,RECORDACRES,0,10;SYSCALACRES "SYSCALACRES" true true false 8 Double 0 0,First,#,VacantUnpermitted,SYSCALACRES,-1,-1;PARCELNUMBER "PARCELNUMBER" true true false 6 Text 0 0,First,#,VacantUnpermitted,PARCELNUMBER,0,6;owner1 "owner1" true true false 45 Text 0 0,First,#,VacantUnpermitted,owner1,0,45;owner2 "owner2" true true false 45 Text 0 0,First,#,VacantUnpermitted,owner2,0,45;owner3 "owner3" true true false 45 Text 0 0,First,#,VacantUnpermitted,owner3,0,45;address "address" true true false 40 Text 0 0,First,#,VacantUnpermitted,address,0,40;city "city" true true false 40 Text 0 0,First,#,VacantUnpermitted,city,0,40;state "state" true true false 2 Text 0 0,First,#,VacantUnpermitted,state,0,2;zip "zip" true true false 5 Text 0 0,First,#,VacantUnpermitted,zip,0,5;plate "plate" true true false 12 Text 0 0,First,#,VacantUnpermitted,plate,0,12;deed "deed" true true false 10 Text 0 0,First,#,VacantUnpermitted,deed,0,10;book "book" true true false 4 Text 0 0,First,#,VacantUnpermitted,book,0,4;page "page" true true false 4 Text 0 0,First,#,VacantUnpermitted,page,0,4;txunit "txunit" true true false 3 Text 0 0,First,#,VacantUnpermitted,txunit,0,3;school "school" true true false 3 Text 0 0,First,#,VacantUnpermitted,school,0,3;legal "legal" true true false 3030 Text 0 0,First,#,VacantUnpermitted,legal,0,3030;PID "PID" true true false 27 Text 0 0,First,#,VacantUnpermitted,PID,0,27;Quickrefid "Quickrefid" true true false 20 Text 0 0,First,#,VacantUnpermitted,Quickrefid,0,20;situs "situs" true true false 38 Text 0 0,First,#,VacantUnpermitted,situs,0,38;AGMAPLINK "AGMAPLINK" true true false 150 Text 0 0,First,#,VacantUnpermitted,AGMAPLINK,0,150;OBJECTID "OBJECTID" true true false 4 Long 0 0,First,#,VacantUnpermitted,OBJECTID,-1,-1;PropertyNumber "PropertyNumber" true true false 255 Text 0 0,First,#,VacantUnpermitted,PropertyNumber,0,255;AdHocTaxYear "AdHocTaxYear" true true false 8 Double 0 0,First,#,VacantUnpermitted,AdHocTaxYear,-1,-1;fpropclass "fpropclass" true true false 255 Text 0 0,First,#,VacantUnpermitted,fpropclass,0,255;FinalLand "FinalLand" true true false 8 Double 0 0,First,#,VacantUnpermitted,FinalLand,-1,-1;FinalBuilding "FinalBuilding" true true false 8 Double 0 0,First,#,VacantUnpermitted,FinalBuilding,-1,-1;FinalTotal "FinalTotal" true true false 8 Double 0 0,First,#,VacantUnpermitted,FinalTotal,-1,-1;flivunits "flivunits" true true false 8 Double 0 0,First,#,VacantUnpermitted,flivunits,-1,-1;Resyearbuilt "Resyearbuilt" true true false 8 Double 0 0,First,#,VacantUnpermitted,Resyearbuilt,-1,-1;commerical_year_built "commerical year built" true true false 255 Text 0 0,First,#,VacantUnpermitted,commerical_year_built,0,255;flbcsact "flbcsact" true true false 8 Double 0 0,First,#,VacantUnpermitted,flbcsact,-1,-1;flbcsfunc "flbcsfunc" true true false 8 Double 0 0,First,#,VacantUnpermitted,flbcsfunc,-1,-1;flbcsfuncsfx "flbcsfuncsfx" true true false 255 Text 0 0,First,#,VacantUnpermitted,flbcsfuncsfx,0,255;flbcsown "flbcsown" true true false 8 Double 0 0,First,#,VacantUnpermitted,flbcsown,-1,-1;flbcssite "flbcssite" true true false 8 Double 0 0,First,#,VacantUnpermitted,flbcssite,-1,-1;JOINPIN_1 "JOINPIN" true true false 255 Text 0 0,First,#,VacantUnpermitted,JOINPIN_1,0,255;Generalization "Generalization" true true false 150 Text 0 0,First,#,VacantUnpermitted,Generalization,0,150;YearBuiltDate "YearBuiltDate" true true false 8 Date 0 0,First,#,VacantUnpermitted,YearBuiltDate,-1,-1;SUM_Area_SQUAREFEET "Summarized area in SQUAREFEET" true true false 8 Double 0 0,First,#,VacantUnpermitted,SUM_Area_SQUAREFEET,-1,-1;Permit "Permit" true true false 50 Text 0 0,First,#,VacantUnpermitted,Permit,0,50;Address_1 "Address" true true false 100 Text 0 0,First,#,VacantUnpermitted,Address_1,0,100;Classification "Classification" true true false 100 Text 0 0,First,#,VacantUnpermitted,Classification,0,100;Submitted "Submitted" true true false 8 Date 0 0,First,#,VacantUnpermitted,Submitted,-1,-1;Issued "Issued" true true false 8 Date 0 0,First,#,VacantUnpermitted,Issued,-1,-1;status_1 "Status" true true false 20 Text 0 0,First,#,VacantUnpermitted,status_1,0,20;Contractor "Contractor" true true false 100 Text 0 0,First,#,VacantUnpermitted,Contractor,0,100;Description "Description" true true false 2147483647 Text 0 0,First,#,VacantUnpermitted,Description,0,2147483647;pid_1 "pid" true true false 255 Text 0 0,First,#,VacantUnpermitted,pid_1,0,255;X "X" true true false 8 Double 0 0,First,#,VacantUnpermitted,X,-1,-1;Y "Y" true true false 8 Double 0 0,First,#,VacantUnpermitted,Y,-1,-1;Xmin "Xmin" true true false 8 Double 0 0,First,#,VacantUnpermitted,Xmin,-1,-1;Xmax "Xmax" true true false 8 Double 0 0,First,#,VacantUnpermitted,Xmax,-1,-1;Ymin "Ymin" true true false 8 Double 0 0,First,#,VacantUnpermitted,Ymin,-1,-1;Ymax "Ymax" true true false 8 Double 0 0,First,#,VacantUnpermitted,Ymax,-1,-1;ARC_Single_Line_Input "Key" true true false 100 Text 0 0,First,#,VacantUnpermitted,ARC_Single_Line_Input,0,100;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,VacantUnpermitted,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,VacantUnpermitted,Shape_Area,-1,-1;PUD_CLASS "PUD_CLASS" true true false 60 Text 0 0,First,#,Lawrence Zoning,PUD_CLASS,0,60;NAME "NAME" true true false 50 Text 0 0,First,#,Lawrence Zoning,NAME,0,50;Zoning "Zoning" true true false 15 Text 0 0,First,#,Lawrence Zoning,Zoning,0,15;GlobalID "GlobalID" false false true 38 GlobalID 0 0,First,#,Lawrence Zoning,GlobalID,-1,-1;OrdNum "Ordinance #" true true false 0 Short 0 0,First,#,Lawrence Zoning,OrdNum,-1,-1;created_user "created_user" false true false 255 Text 0 0,First,#,Lawrence Zoning,created_user,0,255;created_date "created_date" false true false 8 Date 0 0,First,#,Lawrence Zoning,created_date,-1,-1;last_edited_user "last_edited_user" false true false 255 Text 0 0,First,#,Lawrence Zoning,last_edited_user,0,255;last_edited_date "last_edited_date" false true false 8 Date 0 0,First,#,Lawrence Zoning,last_edited_date,-1,-1;Shape__Area "Shape__Area" false true true 0 Double 0 0,First,#,Lawrence Zoning,Shape__Area,-1,-1;Shape__Length "Shape__Length" false true true 0 Double 0 0,First,#,Lawrence Zoning,Shape__Length,-1,-1', "HAVE_THEIR_CENTER_IN", None, '')
arcpy.conversion.FeatureClassToFeatureClass(filegdb+r"\VacantUnpermittedZoning", filegdb, "PotentialGrowthArea", "Zoning IS NOT NULL And NAME IN ('Multi-Dwelling Residential Office', 'Multi-Dwelling Residential', 'Duplex Multi-Dwelling Residential', 'Single-Dwelling Residential', 'Single-Dwelling Residential Office', 'Planned Residential Development', 'Planned Unit Development', 'Planned Commercial Development')", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,VacantUnpermittedZoning,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,VacantUnpermittedZoning,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,VacantUnpermittedZoning,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,VacantUnpermittedZoning,TARGET_FID_1,-1,-1;JOINPIN "JOINPIN" true true false 23 Text 0 0,First,#,VacantUnpermittedZoning,JOINPIN,0,23;RECORDACRES "RECORDACRES" true true false 10 Text 0 0,First,#,VacantUnpermittedZoning,RECORDACRES,0,10;SYSCALACRES "SYSCALACRES" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,SYSCALACRES,-1,-1;PARCELNUMBER "PARCELNUMBER" true true false 6 Text 0 0,First,#,VacantUnpermittedZoning,PARCELNUMBER,0,6;owner1 "owner1" true true false 45 Text 0 0,First,#,VacantUnpermittedZoning,owner1,0,45;owner2 "owner2" true true false 45 Text 0 0,First,#,VacantUnpermittedZoning,owner2,0,45;owner3 "owner3" true true false 45 Text 0 0,First,#,VacantUnpermittedZoning,owner3,0,45;address "address" true true false 40 Text 0 0,First,#,VacantUnpermittedZoning,address,0,40;city "city" true true false 40 Text 0 0,First,#,VacantUnpermittedZoning,city,0,40;state "state" true true false 2 Text 0 0,First,#,VacantUnpermittedZoning,state,0,2;zip "zip" true true false 5 Text 0 0,First,#,VacantUnpermittedZoning,zip,0,5;plate "plate" true true false 12 Text 0 0,First,#,VacantUnpermittedZoning,plate,0,12;deed "deed" true true false 10 Text 0 0,First,#,VacantUnpermittedZoning,deed,0,10;book "book" true true false 4 Text 0 0,First,#,VacantUnpermittedZoning,book,0,4;page "page" true true false 4 Text 0 0,First,#,VacantUnpermittedZoning,page,0,4;txunit "txunit" true true false 3 Text 0 0,First,#,VacantUnpermittedZoning,txunit,0,3;school "school" true true false 3 Text 0 0,First,#,VacantUnpermittedZoning,school,0,3;legal "legal" true true false 3030 Text 0 0,First,#,VacantUnpermittedZoning,legal,0,3030;PID "PID" true true false 27 Text 0 0,First,#,VacantUnpermittedZoning,PID,0,27;Quickrefid "Quickrefid" true true false 20 Text 0 0,First,#,VacantUnpermittedZoning,Quickrefid,0,20;situs "situs" true true false 38 Text 0 0,First,#,VacantUnpermittedZoning,situs,0,38;AGMAPLINK "AGMAPLINK" true true false 150 Text 0 0,First,#,VacantUnpermittedZoning,AGMAPLINK,0,150;OBJECTID "OBJECTID" true true false 4 Long 0 0,First,#,VacantUnpermittedZoning,OBJECTID,-1,-1;PropertyNumber "PropertyNumber" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,PropertyNumber,0,255;AdHocTaxYear "AdHocTaxYear" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,AdHocTaxYear,-1,-1;fpropclass "fpropclass" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,fpropclass,0,255;FinalLand "FinalLand" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,FinalLand,-1,-1;FinalBuilding "FinalBuilding" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,FinalBuilding,-1,-1;FinalTotal "FinalTotal" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,FinalTotal,-1,-1;flivunits "flivunits" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,flivunits,-1,-1;Resyearbuilt "Resyearbuilt" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Resyearbuilt,-1,-1;commerical_year_built "commerical year built" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,commerical_year_built,0,255;flbcsact "flbcsact" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,flbcsact,-1,-1;flbcsfunc "flbcsfunc" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,flbcsfunc,-1,-1;flbcsfuncsfx "flbcsfuncsfx" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,flbcsfuncsfx,0,255;flbcsown "flbcsown" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,flbcsown,-1,-1;flbcssite "flbcssite" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,flbcssite,-1,-1;JOINPIN_1 "JOINPIN" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,JOINPIN_1,0,255;Generalization "Generalization" true true false 150 Text 0 0,First,#,VacantUnpermittedZoning,Generalization,0,150;YearBuiltDate "YearBuiltDate" true true false 8 Date 0 0,First,#,VacantUnpermittedZoning,YearBuiltDate,-1,-1;SUM_Area_SQUAREFEET "Summarized area in SQUAREFEET" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,SUM_Area_SQUAREFEET,-1,-1;Permit "Permit" true true false 50 Text 0 0,First,#,VacantUnpermittedZoning,Permit,0,50;Address_1 "Address" true true false 100 Text 0 0,First,#,VacantUnpermittedZoning,Address_1,0,100;Classification "Classification" true true false 100 Text 0 0,First,#,VacantUnpermittedZoning,Classification,0,100;Submitted "Submitted" true true false 8 Date 0 0,First,#,VacantUnpermittedZoning,Submitted,-1,-1;Issued "Issued" true true false 8 Date 0 0,First,#,VacantUnpermittedZoning,Issued,-1,-1;status_1 "Status" true true false 20 Text 0 0,First,#,VacantUnpermittedZoning,status_1,0,20;Contractor "Contractor" true true false 100 Text 0 0,First,#,VacantUnpermittedZoning,Contractor,0,100;Description "Description" true true false 2147483647 Text 0 0,First,#,VacantUnpermittedZoning,Description,0,2147483647;pid_1 "pid" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,pid_1,0,255;X "X" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,X,-1,-1;Y "Y" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Y,-1,-1;Xmin "Xmin" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Xmin,-1,-1;Xmax "Xmax" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Xmax,-1,-1;Ymin "Ymin" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Ymin,-1,-1;Ymax "Ymax" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Ymax,-1,-1;ARC_Single_Line_Input "Key" true true false 100 Text 0 0,First,#,VacantUnpermittedZoning,ARC_Single_Line_Input,0,100;PUD_CLASS "PUD_CLASS" true true false 60 Text 0 0,First,#,VacantUnpermittedZoning,PUD_CLASS,0,60;NAME "NAME" true true false 50 Text 0 0,First,#,VacantUnpermittedZoning,NAME,0,50;Zoning "Zoning" true true false 15 Text 0 0,First,#,VacantUnpermittedZoning,Zoning,0,15;OrdNum "Ordinance #" true true false 2 Short 0 0,First,#,VacantUnpermittedZoning,OrdNum,-1,-1;created_user "created_user" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,created_user,0,255;created_date "created_date" true true false 8 Date 0 0,First,#,VacantUnpermittedZoning,created_date,-1,-1;last_edited_user "last_edited_user" true true false 255 Text 0 0,First,#,VacantUnpermittedZoning,last_edited_user,0,255;last_edited_date "last_edited_date" true true false 8 Date 0 0,First,#,VacantUnpermittedZoning,last_edited_date,-1,-1;Shape__Area "Shape__Area" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Shape__Area,-1,-1;Shape__Length "Shape__Length" true true false 8 Double 0 0,First,#,VacantUnpermittedZoning,Shape__Length,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,VacantUnpermittedZoning,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,VacantUnpermittedZoning,Shape_Area,-1,-1', '')
arcpy.management.AddField("PotentialGrowthArea", "UnitPotential", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.SelectLayerByAttribute("PotentialGrowthArea", "NEW_SELECTION", "NAME IN ('Single-Dwelling Residential', 'Planned Residential Development', 'Single-Dwelling Residential Office')", None)
arcpy.management.CalculateField("PotentialGrowthArea", "UnitPotential", "1", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute("PotentialGrowthArea", "NEW_SELECTION", "UnitPotential IS NULL", None)
arcpy.management.CalculateField("PotentialGrowthArea", "UnitPotential", "2", "PYTHON3", '', "TEXT")


<Result 'PotentialGrowthArea'>

<Result 'PermittedLandUseUnbuilt'>